In [1]:
## Importing the CSV and creation of a dataframe "poke_df"
import pandas as pd

PATH_POKEDEX_CSV = "./pokedex/pokedex.csv"
poke_df = pd.read_csv(PATH_POKEDEX_CSV)
poke_df = poke_df.drop("Unnamed: 0", axis=1)

In [2]:
## From (STATS1, 5), we learnt about the generations of Pokemon
generations = poke_df["generation"].value_counts().drop_duplicates().sort_index().reset_index()
generations

,generation,count
0,1,192
1,2,107
2,3,165
3,4,121
4,5,171
5,6,85
6,7,99
7,8,88
